In [ ]:
import numpy as np
from PIL import Image
import cv2
import math
from pathlib import Path

In [ ]:
# TEA Algorithm Implementation
def tea_encrypt_words(v0, v1, k):
    delta = 0x9E3779B9
    sum_ = 0
    k0, k1, k2, k3 = k
    for _ in range(32):
        sum_ = (sum_ + delta) & 0xFFFFFFFF
        v0 = (v0 + (((v1 << 4) + k0) ^ (v1 + sum_) ^ ((v1 >> 5) + k1))) & 0xFFFFFFFF
        v1 = (v1 + (((v0 << 4) + k2) ^ (v0 + sum_) ^ ((v0 >> 5) + k3))) & 0xFFFFFFFF
    return v0, v1

def encrypt_bytes_tea(data_bytes, key_words, byteorder='big'):
    n = len(data_bytes)
    pad = (-n) % 8
    if pad:
        data_bytes += b'\x00' * pad

    out = bytearray(len(data_bytes))
    for i in range(0, len(data_bytes), 8):
        block = data_bytes[i:i + 8]
        v0 = int.from_bytes(block[0:4], byteorder)
        v1 = int.from_bytes(block[4:8], byteorder)
        e0, e1 = tea_encrypt_words(v0, v1, key_words)
        out[i:i + 4] = e0.to_bytes(4, byteorder)
        out[i + 4:i + 8] = e1.to_bytes(4, byteorder)

    return bytes(out[:n])

In [ ]:
# Load and preprocess image
Path("TEA Encryption Files").mkdir(parents = True, exist_ok = True)

img_path = r"Enter the path of the input image"
image = Image.open(img_path)
original_image = image.resize((256, 256))
original_image.save("TEA Encryption Files/original_image.png")

# Convert to grayscale for metric uniformity
original_image = cv2.imread("TEA Encryption Files/original_image.png", cv2.IMREAD_GRAYSCALE)
image_array = np.array(original_image)

# Encryption key (128-bit split into 4 words)
key_words = [0x01234567, 0x89ABCDEF, 0xFEDCBA98, 0x76543210]

# Flatten and encrypt
plain_bytes = image_array.tobytes()
cipher_bytes = encrypt_bytes_tea(plain_bytes, key_words, byteorder = 'big')

# Convert back to numpy array and reshape
cipher_array = np.frombuffer(cipher_bytes, dtype = np.uint8)[:image_array.size].reshape(image_array.shape)

# Save encrypted image
encrypted_image = Image.fromarray(cipher_array)
encrypted_image.save("TEA Encryption Files/encrypted_image.png")
encrypted_image = cv2.imread("TEA Encryption Files/encrypted_image.png", cv2.IMREAD_GRAYSCALE)
encrypted_image_path = "TEA Encryption Files/encrypted_image.png"

In [ ]:
# Unified function to calculate metrics
def calculate_metrics(original_image, encrypted_image):
    if original_image is None:
        print("Error: Unable to load original image.")
        return

    # NPCR
    num_changed_pixels = np.sum(original_image != encrypted_image)
    total_pixels = original_image.size
    npcr = (num_changed_pixels / total_pixels) * 100

    # UACI
    abs_diff = np.abs(original_image - encrypted_image)
    uaci = np.mean(abs_diff / 255) * 100

    # Mean Deviation
    mean_deviation = np.mean(np.abs(original_image - encrypted_image))

    # MSE
    mse = np.mean((original_image - encrypted_image) ** 2)

    # PSNR
    max_intensity = 255
    psnr = 10 * np.log10(max_intensity ** 2 / mse)

    # Entropy
    with open(encrypted_image_path, 'rb') as encrypted_image_file:
        encrypted_data = encrypted_image_file.read()

    byte_frequency = [0] * 256
    for byte in encrypted_data:
        byte_frequency[byte] += 1

    entropy = 0
    total_bytes = len(encrypted_data)
    for freq in byte_frequency:
        if freq != 0:
            probability = freq / total_bytes
            entropy -= probability * math.log2(probability)

    return npcr, uaci, mean_deviation, mse, psnr, entropy

In [ ]:
# Metrics calculation
npcr, uaci, mean_deviation, mse, psnr, entropy = calculate_metrics(original_image, encrypted_image)

# Display metrics
print("NPCR score:", npcr)
print("UACI score:", uaci)
print("Mean Deviation:", mean_deviation)
print("Mean Squared Error (MSE):", mse)
print("Peak Signal-to-Noise Ratio (PSNR):", psnr)
print("Entropy of the encrypted image data:", entropy)